In [35]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path
# import logging
# log = logging.getLogger()
# log.setLevel(logging.DEBUG)

from io_utils import *
from peh_utils import * 
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
#### MULTI SESSION


animal_id = 'R600'
bdata = dj.create_virtual_module("new_acquisition", "bdatatest")
subject_session_key = {"ratname": animal_id}

pd_blobs = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids, dates, trials = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

peh_blobs = (bdata.ParsedEvents * bdata.Sessions & subject_session_key).fetch('peh', as_dict=True)

trial_filter = (trials != 0) & (trials != 1)
pd_blobs, peh_blobs, trials, sess_ids, dates = drop_sessions(
    pd_blobs, peh_blobs, trials, sess_ids, dates, trial_filter
)


pd_dicts = convert_to_dict(pd_blobs)
reformat_pd_dict(pd_dicts)

peh_dicts = convert_to_dict(peh_blobs)

assert len(pd_dicts) == len(peh_dicts) == len(trials) == len(sess_ids) == len(dates) , (
    "number of sessions does not match, assumptions of code below are broken!"
)

peh_dicts, peh_dicts, trials, sess_ids, dates = find_and_fix_len_errors(
    pd_dicts, peh_dicts, trials, sess_ids, dates
)


peh_dicts_for_df = get_peh_vars_for_df(peh_dicts, trials, sess_ids)


dropping 1 sessions of 93
dropping 0 sessions of 92


In [3]:

# #### SINGLE SESSION ####
# session_key = {'sessid': 884257}

# prb = (bdata.Sessions & session_key).fetch(
#     "protocol_data", as_dict=True
# )

# s_id, d, t = (bdata.Sessions & session_key).fetch(
#     "sessid", "sessiondate", "n_done_trials"
# )
# prd = convert_to_dict(prb)
# pd_prepare_dict_for_df(prd)
# session_key = {'sessid': s_id[0]}
# pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
# pehd = bt.transform_blob(pehb[0]['peh'])



In [30]:
# #verify
# pd_has_extra_trial = []
# keep_session = []

# for pdd, pehh, trii, sesss in zip(tpd, tpeh, ttri, tsess):
        
#         has_extra_trial, keep = find_and_assess_ntrial_mismatches(
#             pdd, pehh, trii, sesss
#         )
#         pd_has_extra_trial.append(has_extra_trial)
#         keep_session.append(keep)

# np.sum(pd_has_extra_trial), np.sum(keep_session)


(0, 89)

In [64]:
pd.concat([pd_df,peh_df],axis=1)

,hits,temperror,sides,result,sa,sb,delay,fixation,timeouts,helper,stage,is_match,early_spoke,t_start,t_end,cpoke_in,cpoke_out,spoke_in
0,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,67.397704,93.532804,87.847704,87.993804,88.216004
1,0.0,0.0,l,2.0,12000.0,12000.0,0.15,0.55,5.0,0.0,8.0,True,1.0,99.722304,110.046404,104.366804,104.479804,104.655604
2,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,111.174604,116.927804,111.267804,111.451204,111.647504
3,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,118.505004,133.339004,127.613604,127.728104,127.926004
4,0.0,0.0,l,2.0,3000.0,3000.0,0.15,0.55,5.0,0.0,8.0,True,0.0,134.797304,145.090404,139.275204,139.384604,139.867004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.0,0.0,l,1.0,3000.0,3000.0,0.15,0.55,NaN,0.0,8.0,True,1.0,3840.883503,3845.851003,3843.117303,3843.287603,3843.450503
330,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,3849.663203,3856.254603,3850.441703,3850.675703,3850.786103
331,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,3856.820703,3866.590003,3860.790303,3860.967503,3861.110703
332,0.0,0.0,r,2.0,3000.0,12000.0,0.15,0.55,5.0,0.0,8.0,False,1.0,3866.918503,3876.796203,3870.894103,3871.044703,3871.221103


In [13]:
a = np.array([])
a.size

0

In [18]:
peh_dicts[1][0]['states']

{'state_0': array([[       nan, 716.268904],
        [731.136304,        nan]]),
 'check_next_trial_ready': array([[       nan, 716.269004],
        [731.136304, 731.136304]]),
 'wait_for_spoke': array([[716.268904,        nan],
        [716.269004, 725.093704]]),
 'left_hit_state': array([], shape=(0, 2), dtype=float64),
 'right_hit_state': array([725.093704, 725.136004]),
 'left_drink_state': array([], shape=(0, 2), dtype=float64),
 'right_drink_state': array([725.136004, 730.136004]),
 'hit_state': array([730.136004, 730.136304]),
 'general_final_state': array([730.136304, 731.136304]),
 'starting_state': 'state_0',
 'ending_state': 'state_0'}

In [43]:

peh_for_df = get_peh_vars_for_df(peh_dicts, trials, sess_ids)

879474 0
0
630.0812040000001
valid multi trial
1
1306.419304
valid multi trial
2
1334.2636670000002
valid multi trial
3
1406.1541679999998
valid multi trial
4
2227.558469
valid multi trial
5
2268.034865
valid multi trial
6
2497.224166
valid multi trial
7
valid trial
8
2626.1667660000003
valid multi trial
9
valid trial
10
2994.190466
valid multi trial
11
3107.074166
valid multi trial
12
valid trial
13
3373.046167
valid multi trial
14
valid trial
15
valid trial
16
valid trial
17
valid trial
18
3556.104966
valid multi trial
19
3723.369266
valid multi trial
20
3776.8900670000003
valid multi trial
21
valid trial
22
3947.875003
valid multi trial
23
valid trial
24
valid trial
25
valid trial
26
valid trial
27
valid trial
879662 1
0
725.093704
valid multi trial
1
valid trial
2
valid trial
3
valid trial
4
valid trial
5
valid trial
6
valid trial
7
valid trial
8
valid trial
9
valid trial
10
valid trial
11
valid trial
12
valid trial
13
valid trial
14
valid trial
15
valid trial
16
valid trial
17
val

In [45]:
len(peh_for_df)


90

In [41]:
fetch_first_spoke(dd)

(3038.394803, True)

In [31]:
dd['wait_for_spoke'].size

dd['wait_for_spoke'][-1][1]

3038.4466030000003

In [39]:
dd['early_spoke_state'][-1][0]

3038.394803